## Factors against Parallelism

* Startup costs associated with initiating (or tearing down) processes
  * May often overwhelm actual processing time (rendering ||ism useless)
  * Involve thread/process creation, data movement
* Interference: slowdown resulting from multiple processors accessing shared resources
  * Resources: memory, I/O, system bus, sub-processors
  * Software synchronization: locks, latches, mutexes
  * Hardware synchronization: cache faults, interrupts
* Skew: when breaking a single task into many smaller tasks, not all tasks may be the same size
  * Not all tasks finish at the same time
  
The course will cover all of these in specific details. At this time, you should understand these three factors by visual example. I used to teach this unit by analogy with the real world examples. This was cute, but confusing. I've left this material at the end. The visual examples are boring, but concise and accurate.

#### Startup Costs

The NERSC tutorial on OpenMP (our next unit) draws a schematic of a parallel program as consisting of parallel execution parts and serial parts. 

<img src="https://docs.nersc.gov/development/programming-models/openmp/OpenMPforkjoin.png" width="500" title="https://docs.nersc.gov/development/programming-models/openmp/OpenMPforkjoin.png" />

The portions of the progam in the parallel region during which the threads are not running fully in parallel are the startup costs (or teardown costs). The dashed lines indicate periods of time in which the program is not running in parallel. The solid lines indicate times when the program is running in parallel.

During the sequential parts, one would not expect parallelism. This diagram shows that even during the parallel parts, one might not realize perfect parallelism.

Examples of startup costs include:
  * operating system overheads: creation of threads of processes
  * data access: loading or copying data to the parallel workers
  
Similarly we may experience teardown costs in closing parallel contexts:
  * operating system cleaning up thread/process state
  * copying partial results from parallel computation to memory or storage
  

#### Interference

The paper [Thiffault et al. Dynamic instrumentation of large-scale MPI and OpenMP applications](http://cs.umanitoba.ca/pub/IPDPS03/DATA/16_04_089.PDF) shows a timeline of parallel computation of a neutron-transfer physics code. The image shows multiple processes awaiting on data to be transferred from other nodes and threads within each process executing intermittently.   

<img src="./images/sweep3d.png" width="500" /> 

Interference arises when parallel execution contexts have to wait on each other. This manifests in multiple ways:
  * communicating processes waiting on messages
  * processes waiting on shared resources
      * these can be sofware constructs, such as locks
      * or they can be hardware constructs, such as cache lines


### Skew

Skew arises when execution cannot advance until all jobs complete. This problem is most acute when one decomposes a problem into independent parts that are of different sizes. The following chart from [Uselton et al. Parallel I/O Performance: From Events to Ensembles](https://crd.lbl.gov/assets/pubs_presos/CDS/FTG/Papers/2010/ipdps10ipm.pdf) shows a parallel code that is running particularly poorly owing to long-running processes.

<img src="./images/ioensembles.png" width="500" />

This is an I/O trace. The blue and red lines indicate periods when I/O is being conducted and the white space indicates all else. In this code, all I/O must complete prior to a barrier prior to the computation continuing. The chart shows processes completing waiting for the long running processes. 

Long running processes are called. "stragglers".

#### Multiple Factors

Many computational frameworks operate in a manner in which they:
  * launch a bunch of parallel jobs (startup)
  * wait for all jobs to complete (skew)
  * integrate/coordinate the parts (interference)
  * restart a bunch of new parallel jobs -- REPEAT!
  
  
The following diagram of the bulk synchronous computing model shows skew and synchronization. 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ee/Bsp.wiki.fig1.svg/540px-Bsp.wiki.fig1.svg.png" width="500" />

### In Computers: Real things that Degrade Parllelism

* I/O (memory and storage)
  * may be startup (load data before computation)
  * may be interference (awaiting data transfer between parallel tasks)
  * may be skew (await I/O completion of one task)
* Network communication
    * similar to I/O but always involves communication
* Failures—particularly slow/failed processes (often skew)

The HPC community focuses on communication (among processes) as the major source of slowdown.  This is a traditional (I/O and networking) view.

### Communication

* Parallel computation proceeds in phases
  * Compute (evaluate data that you have locally)
* Communicate (exchange data among compute tasks).  Performance is governed by:
  * Latency: fixed cost to send a message
  * Round trip time (speed of light and switching costs)
* Bandwidth: marginal cost to send a message
  * Link capacity
* Latency dominates small messages and bandwidth dominates large.
  * It is lmost always better to increase message size for performance, but difficult to achieve in practice.

### Mitigation: Overlapped I/O and Computation

(I/O or messaging) and computation that occur in parallel are overlapped

<img src="./images/overlap.png" width="512" title="Unknown source" />

* _Concept_: When performing a slow operation
  * do the slow operation asynchronously
  * do useful work with processor while waiting
* Overlap is one of the simplest and most important forms of asynchronous execution
  * identify independent tasks and do in parallel
  * reorder I/O to initiate as early as possible and wait as late as possible
  * while computing at the same time
  
I've built a toy example to demonstrate.

In [1]:
# compute 
def factorial(number):  
    f = 1
    for i in range(2, number+1):
        f *= i
    return f
 
# synchronous I/O
def io_from_devnull(number):
    with open("/dev/null", "rb") as fh:
        for i in range(number):
            fh.read(1)
    return number

In [2]:
%timeit factorial(10000)
%timeit io_from_devnull(30000)

23.4 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
22 ms ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit -n 20 
factorial(10000)
io_from_devnull(30000)

45.4 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [2]:
%%timeit -n 20

import paralleldefs

from multiprocessing import Process
p1 = Process(target=paralleldefs.factorial, args=(10000,))
p2 = Process(target=paralleldefs.io_from_devnull, args=(30000,))
p1.start()
p2.start() 
p1.join()
p2.join()

69.2 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


### Factors Conclusions

* Factors against parallelism are the most important design consideration.
* Considered in light of Amdahl's law
    * when we estimate an Amdahl number from empirical results all factors contribute to the unoptimized portion
    * when we implement a parallel portion from an instrumented serial version, the factors are why the realized speedup is less than the ideal speedup
